In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
from sklearn.linear_model import LassoCV
import lightgbm as lgb

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X, y)

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024908851921949804, tolerance: 0.02152694805194805
  model = cd_fast.enet_coordinate_descent_gram(
/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06303854642465012, tolerance: 0.021966098945660997
  model = cd_fast.enet_coordinate_descent_gram(
/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.072116817079106, tolerance: 0.021966098945660997
  model = cd_fast.enet_coordinate_descent_gram(
/Users/prathmesh/a

LassoCV(cv=5, random_state=42)

In [8]:
selected_mask = lasso.coef_ != 0
selected_features = list(X.columns[selected_mask])
print("Selected Features:", selected_features)

Selected Features: ['AMHng_mL', 'Follicle_No_R', 'Skin_darkening_Y_N', 'Fast_food_Y_N', 'Follicle_No_L', 'CycleR_I', 'Menstrual_Irregularity', 'Antral_Follicle_Count', 'LHmIU_mL', 'hair_growthY_N', 'HeightCm', 'Weight_gainY_N', 'Blood_Group', 'BMI', 'Testosterone_Levelng_dL']


In [9]:
X = X[selected_features]
X

,AMHng_mL,Follicle_No_R,Skin_darkening_Y_N,Fast_food_Y_N,Follicle_No_L,CycleR_I,Menstrual_Irregularity,Antral_Follicle_Count,LHmIU_mL,hair_growthY_N,HeightCm,Weight_gainY_N,Blood_Group,BMI,Testosterone_Levelng_dL
0,-0.906849,-1.121365,-0.987716,1.309132,-1.003230,-0.956882,0.656627,-0.416873,-0.021232,-0.944133,-1.084917,-1.087201,0.834437,-1.339000,0.920608
1,-1.034602,-0.494497,-0.987716,-1.445336,-1.003230,-0.956882,-0.744197,-0.551280,-0.068914,-0.944133,1.088646,-1.087201,0.834437,-0.159445,-0.684068
2,0.171947,2.639841,-0.987716,1.309132,2.299153,-0.956882,1.123568,0.154359,-0.072780,-0.944133,1.889432,-1.087201,-2.356302,-0.075192,0.412105
3,-1.107941,-1.434799,-0.987716,-1.445336,-1.333468,-0.956882,0.656627,-0.215261,-0.045533,-0.944133,-2.000102,-1.087201,-0.760932,0.851601,-0.415100
4,-0.861900,-0.807931,-0.987716,-1.445336,-1.003230,-0.956882,-0.277256,-0.248863,-0.072412,-0.944133,0.974248,-1.087201,-2.356302,-1.170492,-0.029409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,-0.137971,-0.369124,0.197697,-0.894443,-0.672992,-0.956882,1.123568,0.927203,-0.046490,-0.309909,-0.124340,-1.087201,1.153511,-1.528571,1.824951
1537,-0.601190,-0.181063,-0.395009,-1.445336,0.251675,-0.355268,1.123568,-1.760948,-0.054922,-0.309909,-1.130676,0.454084,0.355826,0.683094,-1.585366
1538,-0.146014,-0.369124,-0.987716,-0.343549,-0.144610,-0.355268,-1.211138,1.767250,-0.064017,-0.309909,-0.307010,-0.059678,-0.122784,0.556713,-1.387446
1539,0.041829,-0.807931,0.790403,1.309132,-0.342753,0.847959,-1.211138,-1.424929,-0.022226,-0.309909,0.530566,1.481607,0.515363,0.346078,1.819876


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [11]:
def objective(trial):
    # Suggest hyperparameters to try
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "objective": "binary",  # or "multiclass" if you're doing that
        "verbose": -1,
    }


    train_data = lgb.Dataset(x_train, label=y_train)
    valid_data = lgb.Dataset(x_test, label=y_test)

    gbm = lgb.train(param,
                    train_data,
                    valid_sets=[valid_data],
                    num_boost_round=1000)

    y_pred = gbm.predict(x_test)
    y_pred_labels = (y_pred > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred_labels)

    return 1.0 - acc  # Optuna minimizes, so 1 - accuracy


In [12]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 21:11:05,748] A new study created in memory with name: no-name-87efded8-6133-44e2-8041-2c8aed7eafd9
[I 2025-05-31 21:11:06,777] Trial 0 finished with value: 0.030172413793103425 and parameters: {'learning_rate': 0.13853854829887755, 'n_estimators': 853, 'max_depth': 8, 'num_leaves': 108, 'min_child_samples': 5, 'subsample': 0.7664635939472395, 'colsample_bytree': 0.7953624881481882, 'reg_alpha': 4.90797894791463e-08, 'reg_lambda': 1.5949980327169552e-05}. Best is trial 0 with value: 0.030172413793103425.
[I 2025-05-31 21:11:08,240] Trial 1 finished with value: 0.030172413793103425 and parameters: {'learning_rate': 0.06297868291382412, 'n_estimators': 979, 'max_depth': 7, 'num_leaves': 50, 'min_child_samples': 5, 'subsample': 0.9681386715367346, 'colsample_bytree': 0.8970711263000706, 'reg_alpha': 6.755352956950888e-08, 'reg_lambda': 0.00023791174704688097}. Best is trial 0 with value: 0.030172413793103425.
[I 2025-05-31 21:11:11,050] Trial 2 finished with value: 0.0301724

Best hyperparameters:
{'learning_rate': 0.16288003773058532, 'n_estimators': 491, 'max_depth': 10, 'num_leaves': 101, 'min_child_samples': 34, 'subsample': 0.7296080323099666, 'colsample_bytree': 0.9697687556927292, 'reg_alpha': 6.057213445209071e-08, 'reg_lambda': 1.285342245909127e-08}
Best accuracy:
0.9741379310344828


In [13]:
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_test, label=y_test)

best_model = lgb.train(study.best_params,
                train_data,
                valid_sets=[valid_data],
                num_boost_round=1000)

# Predict
preds = best_model.predict(x_test)
preds = (preds > 0.5).astype(int)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")



Final evaluation on test set:
Accuracy: 97.41
Precision: 94.74
Recall: 94.74
F1 Score: 94.74
